In [57]:
from dash import Dash, dcc, html, Input, Output
import plotly.express as px
import io
import os
import netCDF4
import xarray as xr
import plotly.graph_objects as go
import pandas as pd
pd.options.mode.copy_on_write = True

f9_1480_1520 = '/Users/madison.soden/Desktop/1480_1520_NN_ODV7901009QC.TXT'
#f9_1100_1200 = '/Users/madison.soden/Desktop/1100_1200_NN_ODV7901009QC.TXT'
#f9_1100_1520 = '/Users/madison.soden/Desktop/1100_1520_NN_ODV7901009QC.TXT'
#f9_raw = '/Users/madison.soden/Desktop/ODV7901009.TXT'
f22 = '/Users/madison.soden/BGCARGO_work/data/4903622/ODV4903622QC.TXT'
netcdf_dir = '/Users/madison.soden/BGCARGO_work/netcdf_holder_04_25/4903622/' 


In [58]:
def get_netcdf_files(directory):
        # Get all files and directories in the specified directory
        files = os.listdir(directory)
        # Filter out directories, only keep files
        files = [f for f in files if os.path.isfile(os.path.join(directory, f))]
        return files

In [59]:
def xarray_netcdf_df( filename): 
    ds = xr.open_dataset(filename, drop_variables=["DATA_TYPE", "FORMAT_VERSION", "HANDBOOK_VERSION", "HISTORY_SOFTWARE", "HISTORY_SOFTWARE_RELEASE", 
                                                   "HISTORY_REFERENCE", "HISTORY_DATE", "REFERENCE_DATE_TIME", "DATE_CREATION", "DATE_UPDATE", "HISTORY_ACTION",
                                                   "HISTORY_QCTEST", "HISTORY_START_PRES", "HISTORY_STOP_PRES", "PI_NAME", "PROJECT_NAME", "SCIENTIFIC_CALIB_COEFFICIENT"
                                                   "SCIENTIFIC_CALIB_EQUATION", 'PLATFORM_TYPE', 'FLOAT_SERIAL_NO', 'FIRMWARE_VERSION', 'WMO_INST_TYPE', 'JULD', 
                                                   'JULD_QC', 'JULD_LOCATION', 'LATITUDE', 'LONGITUDE', 'POSITION_QC', 'POSITIONING_SYSTEM', 'VERTICAL_SAMPLING_SCHEME',
                                                   'DIRECTION', 'DATA_CENTRE', 'DC_REFERENCE', 'SCIENTIFIC_CALIB_EQUATION', 'SCIENTIFIC_CALIB_COEFFICIENT', 'SCIENTIFIC_CALIB_COMMENT', 
                                                   'SCIENTIFIC_CALIB_DATE', 'HISTORY_INSTITUTION', 'HISTORY_STEP', 'HISTORY_PARAMETER', 'HISTORY_PREVIOUS_VALUE'])
    df = ds.to_dataframe()
    df = df.reset_index()
    df1 = df[['PRES', 'DOXY_ADJUSTED', 'DOXY_ADJUSTED_QC', 'BBP700_ADJUSTED', 'BBP700_ADJUSTED_QC', 'CHLA_ADJUSTED', 'CHLA_ADJUSTED_QC', 
               'NITRATE_ADJUSTED', 'NITRATE_ADJUSTED_QC']].copy() #'DOXY_ADJUSTED_ERROR', 'BBP700_ADJUSTED_ERROR', 'CHLA_ADJUSTED_ERROR', 'NITRATE_ADJUSTED_ERROR', 
    #print(list(df1.columns))
    df1 = df1.dropna(subset=["DOXY_ADJUSTED"])
    for c in ['DOXY_ADJUSTED_QC', 'BBP700_ADJUSTED_QC', 'CHLA_ADJUSTED_QC', 'NITRATE_ADJUSTED_QC']: 
        df1[c] = df1[c].apply(lambda x: x.decode('utf-8'))
    df1 = df1.drop_duplicates()
    df1.rename(columns={'BBP700_ADJUSTED':'BBP_700[1/m]', 'CHLA_ADJUSTED':'CHLA[mg/m^3]', 'NITRATE_ADJUSTED':'Nitrate[µmol/kg]', 'DOXY_ADJUSTED':'Oxygen[µmol/kg]',
                        'DOXY_ADJUSTED_QC':'OxygenQF', 'BBP700_ADJUSTED_QC':'BBPQF', 'CHLA_ADJUSTED_QC':'CHLAQF', 'NITRATE_ADJUSTED_QC':'NitrateQF', 'PRES':'Depth[m]' }, inplace=True)
    df1 = df1.reset_index(drop=True)
    return df1

In [68]:
files = get_netcdf_files(netcdf_dir)
#print(files)
#['AOML_BR4903622_122.nc', 'AOML_BR4903622_116.nc', 'AOML_BR4903622_123.nc', 'AOML_BR4903622_117.nc', 'AOML_BR4903622_118.nc', 'AOML_BR4903622_119.nc', 'AOML_BR4903622_120.nc',
# 'AOML_BR4903622_114.nc', 'AOML_BR4903622_124.nc', 'AOML_BR4903622_125.nc', 'AOML_BR4903622_121.nc', 'AOML_BR4903622_115.nc']
df = pd.DataFrame()
for f in files:
    df1 = xarray_netcdf_df(netcdf_dir+f)
    df1['Station'] = int(f[-6:-3])
    df1.loc[len(df)] = [None] * len(df1.columns)
    df = pd.concat([df,df1], axis=0)
print(df)
df_Amode22 = df

/var/folders/vn/tqwq9q1d0kq4yyrs939xj72dfq_zkx/T/ipykernel_1612/1778740913.py:9: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

/var/folders/vn/tqwq9q1d0kq4yyrs939xj72dfq_zkx/T/ipykernel_1612/1778740913.py:9: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

/var/folders/vn/tqwq9q1d0kq4yyrs939xj72dfq_zkx/T/ipykernel_1612/1778740913.py:9: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA 

        Depth[m]  Oxygen[µmol/kg] OxygenQF  BBP_700[1/m] BBPQF  CHLA[mg/m^3]  \
0            NaN              NaN     None           NaN  None           NaN   
1      11.690000       201.193253        1      0.001259     1       0.24455   
2      16.530001       201.231186        1      0.001203     1       0.22630   
3      21.770000       201.283752        1      0.001436     1       0.23360   
4      26.719999       201.242889        1      0.001303     1       0.23725   
..           ...              ...      ...           ...   ...           ...   
65   1599.390015       203.005234        1      0.000323     1       0.00365   
66   1699.589966       203.871933        1      0.000257     1      -0.00730   
67   1800.300049       204.410065        1      0.000279     1      -0.00730   
68   1899.760010       204.775436        1      0.000290     1       0.00365   
769          NaN              NaN     None           NaN  None           NaN   

    CHLAQF  Nitrate[µmol/kg] NitrateQF 

/var/folders/vn/tqwq9q1d0kq4yyrs939xj72dfq_zkx/T/ipykernel_1612/1778740913.py:9: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



In [69]:
def det_skiprows_odvqc(filename):
    '''This function determines the number of header lines (starting with "//")
    in and ODV{floatid}QC.txt file with the given filename.
    Returns the number of header lines. 
    Hartmut Frenzel'''
    with open(filename, 'r') as f_in:
        lines = f_in.readlines()
    skiprows = 0
    nit_break_points = []
    for line in lines:
        if line[0:2] == '//':
            if line.startswith("//Nitrate"):
                Nit_coefs = line.split("\t")
                nit_break_points.append(int(Nit_coefs[1]))
            skiprows += 1
        else:
            break
    return skiprows, nit_break_points

In [70]:
def load_ODV( fileName):
    #loading relevant odvfile into pandas array
    skip, nit_break_points=det_skiprows_odvqc(fileName)
    df_ODV = pd.read_csv( fileName, sep='\t', skiprows=skip)
    
    df_ODV = df_ODV.reset_index(drop=True)
    df_ODV = df_ODV.rename(columns={"QF.1": "PressureQF", "QF.2": "TemperatureQF", "QF.3": "SalinityQF", 
                                    "QF.6": "OxygenQF", "QF.8": "NitrateQF", "QF.9": "CHLAQF", "QF.10": "BBPQF", "QF.12": "pH_QF"})
        # df_ODV = df_ODV.iloc[:, [ 1, 2, 3, 4, 8, 9, 10, 11, 12, 13, 16, 18, 19, 22, 23, 24, 25, 26, 27, 30, 31]]
        # pressure = 8, pressureQF = 9, Temp = 10, TempQF = 11, Salinity =12, SalinityQF= 13
        # depth = 16, Oxygen =18, oxygenQF = 19, nitrate = 22, nitrateQF = 23, 
        # Chl_a = 24, Chl_aQF = 25, BBP700 = 26, BBPQF = 27, pH = 30, pHQF = 31
    
        #foo = str(df_ODV.iloc[5, 'mon/day/yr']) + " " + str(df_ODV.iloc[5, 'hh:mm'])
    df_ODV.rename(columns={'b_bp700[1/m]':'BBP_700[1/m]'}, inplace=True)
    df_ODV.rename(columns={'Chl_a[mg/m^3]':'CHLA[mg/m^3]'}, inplace=True)
    df_ODV.rename(columns={'pHinsitu[Total]':'pH_Insitu_total'}, inplace=True)
    for i in df_ODV.index:
        df_ODV.loc[i, 'mon/day/yr'] = pd.to_datetime( (str(df_ODV['mon/day/yr'][i]) + " " + str(df_ODV['hh:mm'][i])), format='%m/%d/%Y %H:%M')
    
    for i in df_ODV.index: 
        if( df_ODV.loc[i, 'Nitrate[µmol/kg]'] == -10000000000.0 ):
            df_ODV.loc[i, 'Nitrate[µmol/kg]'] = None
            df_ODV.loc[i, 'NitrateQF'] = None
        if( df_ODV.loc[i, 'Oxygen[µmol/kg]'] == -10000000000.0 ):
            df_ODV.loc[i, 'Oxygen[µmol/kg]'] = None
            df_ODV.loc[i, 'Oxygen[µmol/kg]'] = None
        if( df_ODV.loc[i, 'BBP_700[1/m]'] == -10000000000.0 ):
            df_ODV.loc[i, 'BBP_700[1/m]'] = None
            df_ODV.loc[i, 'BBP_700[1/m]'] = None
        if( df_ODV.loc[i, 'CHLA[mg/m^3]'] == -10000000000.0 ):
            df_ODV.loc[i, 'CHLA[mg/m^3]'] = None
            df_ODV.loc[i, 'CHLA[mg/m^3]'] = None
            #df_ODV = df_ODV.loc[df_ODV['Oxygen[µmol/kg]'] != -10000000000.0]
            #df_ODV = df_ODV.loc[df_ODV.iloc[:,23] != 1] Remove Nitrate values of QC = bad
            #df_ODV = df_ODV.dropna()
        
    df_Nit = df_ODV[df_ODV['Nitrate[µmol/kg]'].notna()]
    # used the following for loop instead of ^.notna()]^ apprach to allow myself to retain 
    # the 'None'values inbetween profiles or 'Station' changes. So that the scatter plots 
    # trace correctly. This is a temp solution. I should probably split by station in the 
    # future but I'm slightly concerned about processing time when regraphing. 
    
        #for i in df_Nit.index[:-1]: 
        #    if( (df_Nit.loc[i, 'Station'] == df_Nit.loc[i+1, 'Station']) and 
        #        (df_Nit.loc[i, 'Nitrate[µmol/kg]'] == None) ):
        #        df_Nit.drop([i])
        #print(df_Nit)
    
    df_Nit['NitrateQF'] = df_Nit['NitrateQF'].apply(str)
    df_Nit['OxygenQF'] = df_Nit['OxygenQF'].apply(str)
    df_Nit['BBPQF'] = df_Nit['BBPQF'].apply(str)
    df_Nit['CHLAQF'] = df_Nit['CHLAQF'].apply(str)
    df_Nit = df_Nit.reset_index(drop=True)
    
    return df_Nit, nit_break_points#.groupby("Station", as_index=False) #removed list(kjdfjsdf)

In [71]:
df_9, nit_break_points_9 = load_ODV(f9_1480_1520)
df_9_grouped = df_9.groupby("Station", as_index=False)

df_22, nit_break_points_22 = load_ODV(f22)
df_22_grouped = df_22.groupby("Station", as_index=False) 


f9_nprof = len(df_9_grouped)+1 #will return 49 for a float w/ 49 profiles
f22_nprof = len(df_22_grouped)+2
max_nprof = max(f9_nprof, f22_nprof)
#max_nprof = f22_nprof

line_colors_greens = [ '#1aa000', '#25e200', '#79ff5e'] #greens 
line_colors_reds = [ '#b22222', '#e65c00', '#ff8c42'] #reds 
line_colors_blues = [ '#007bff', '#5d9eff', '#87c4ff'] #blues
line_colors_greys = ['#494541', '#696969', '#9f9f9f'] #greyscale
#px.colors.qualitative.Set3
#px.colors.sequential.Viridis_r

In [72]:
print(df_22.columns.tolist())
#['Cruise', 'Station', 'Type', 'mon/day/yr', 'hh:mm', 'Lon [°E]', 'Lat [°N]', 'QF', 'Pressure[dbar]', 'PressureQF', 'Temperature[°C]', 'TemperatureQF', 'Salinity[pss]', 'SalinityQF', 'Sigma_theta[kg/m^3]', 'QF.4', 'Depth[m]', 'QF.5', 'Oxygen[µmol/kg]', 'OxygenQF', 'OxygenSat[%]', 'QF.7', 'Nitrate[µmol/kg]', 'NitrateQF', 'CHLA[mg/m^3]', 'CHLAQF', 'BBP_700[1/m]', 'BBPQF', 'CDOM[ppb]', 'QF.11', 'pH_Insitu_total', 'pH_QF', 'CP660[1/m]', 'QF.13']

['Cruise', 'Station', 'Type', 'mon/day/yr', 'hh:mm', 'Lon [°E]', 'Lat [°N]', 'QF', 'Pressure[dbar]', 'PressureQF', 'Temperature[°C]', 'TemperatureQF', 'Salinity[pss]', 'SalinityQF', 'Sigma_theta[kg/m^3]', 'QF.4', 'Depth[m]', 'QF.5', 'Oxygen[µmol/kg]', 'OxygenQF', 'OxygenSat[%]', 'QF.7', 'Nitrate[µmol/kg]', 'NitrateQF', 'CHLA[mg/m^3]', 'CHLAQF', 'BBP_700[1/m]', 'BBPQF', 'CDOM[ppb]', 'QF.11', 'pH_Insitu_total', 'pH_QF', 'CP660[1/m]', 'QF.13']


In [73]:
#df_Amode22
df_Amode22_grouped = df_Amode22.groupby("Station", as_index=False)

In [74]:
for r in range(112,125):
    df_22.loc[len(df_22)] = [None] * len(df_22.columns)
    df_22.loc[len(df_22)-1, 'Station'] = r

/var/folders/vn/tqwq9q1d0kq4yyrs939xj72dfq_zkx/T/ipykernel_1612/3811073414.py:2: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

/var/folders/vn/tqwq9q1d0kq4yyrs939xj72dfq_zkx/T/ipykernel_1612/3811073414.py:2: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

/var/folders/vn/tqwq9q1d0kq4yyrs939xj72dfq_zkx/T/ipykernel_1612/3811073414.py:2: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA 

In [79]:
for r in range(1,114):
    df_Amode22.loc[len(df_Amode22)] = [None] * len(df_Amode22.columns)
    df_Amode22.loc[len(df_Amode22)-1, 'Station'] = r

/var/folders/vn/tqwq9q1d0kq4yyrs939xj72dfq_zkx/T/ipykernel_1612/2503609621.py:2: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

/var/folders/vn/tqwq9q1d0kq4yyrs939xj72dfq_zkx/T/ipykernel_1612/2503609621.py:2: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

/var/folders/vn/tqwq9q1d0kq4yyrs939xj72dfq_zkx/T/ipykernel_1612/2503609621.py:2: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA 

In [75]:
#foo_row = pd.DataFrame({'Station': [112], 'Cruise':None, 'Type':None, 'mon/day/yr':None, 'hh:mm':None, 'Lon [°E]':None, 'Lat [°N]':None, 'QF':None, 'Pressure[dbar]':None, 'PressureQF':None,
#                        'Temperature[°C]':None, 'TemperatureQF':None, 'Salinity[pss]':None, 'SalinityQF':None, 'Sigma_theta[kg/m^3]':None, 'QF.4':None, 'Depth[m]':None, 'QF.5':None, 
#                        'Oxygen[µmol/kg]':None, 'OxygenQF':None, 'OxygenSat[%]':None, 'QF.7':None, 'Nitrate[µmol/kg]':None, 'NitrateQF':None, 'CHLA[mg/m^3]':None, 'CHLAQF':None, 'BBP_700[1/m]':None,
#                        'BBPQF':None, 'CDOM[ppb]':None, 'QF.11':None, 'pH_Insitu_total':None, 'pH_QF':None, 'CP660[1/m]':None, 'QF.13':None})
#df_22 = pd.concat([df, new_row], ignore_index=True)
df_22.loc[len(df_22)] = [None] * len(df_22.columns)
df_22.loc[len(df_22)-1, 'Station'] = 112
print(df_22)

       Cruise Station  Type           mon/day/yr  hh:mm  Lon [°E]  Lat [°N]  \
0     4903622       1     C  2021-10-05 21:49:00  21:49   -93.388    24.797   
1     4903622       1     C  2021-10-05 21:49:00  21:49   -93.388    24.797   
2     4903622       1     C  2021-10-05 21:49:00  21:49   -93.388    24.797   
3     4903622       1     C  2021-10-05 21:49:00  21:49   -93.388    24.797   
4     4903622       1     C  2021-10-05 21:49:00  21:49   -93.388    24.797   
...       ...     ...   ...                  ...    ...       ...       ...   
7502     None     121  None                 None   None       NaN       NaN   
7503     None     122  None                 None   None       NaN       NaN   
7504     None     123  None                 None   None       NaN       NaN   
7505     None     124  None                 None   None       NaN       NaN   
7506     None     112  None                 None   None       NaN       NaN   

        QF  Pressure[dbar]  PressureQF  ...  CHLA[m

/var/folders/vn/tqwq9q1d0kq4yyrs939xj72dfq_zkx/T/ipykernel_1612/100705785.py:6: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



In [76]:
marks_9 = {x:{'label': str(x), 'style': {'color': line_colors_greens[0]}} for x in nit_break_points_9}
marks_22 = {x:{'label': str(x), 'style': {'color': line_colors_blues[0]}} for x in nit_break_points_22}
marks = marks_9 | marks_22
#marks = marks_22
#marks.update({max_nprof:{'label': str(max_nprof)}})
marks = dict(sorted(marks.items()))

In [83]:
app = Dash(__name__)

#buffer = io.StringIO()
#fig.write_html(buffer)

#html_bytes = buffer.getvalue().encode()
#encoded = b64encode(html_bytes).decode()


app.layout = html.Div([
    html.Div([
        html.H5("Variable"),
            dcc.Dropdown(
                id="n_param",
                options=['Oxygen[µmol/kg]', 'Nitrate[µmol/kg]', 'pHinsitu[Total]', 'BBP_700[1/m]', 'CHLA[mg/m^3]'],
                value='CHLA[mg/m^3]',
            ),    
        html.H5("Filter by Cycle #:"),
        dcc.RangeSlider(
            id='range-slider',
            min=1, max=125, step=1,
#                        min=1, max=max_nprof, step=1,
            marks= marks,
            value=[34, 35],
            tooltip={
            "always_visible": True,
            "template": "{value}"
            },
        )
    ], style={'width': '49%', 'display': 'inline-block'}),
    html.Div([
        dcc.Graph(id="scatter-plot")
        ], style={'height': '49%'}) #'display': 'inline-block'
])

@app.callback( 
    Output("scatter-plot", "figure"), 
    Input("range-slider", "value"),
    Input("n_param", "value"))
def update_bar_chart(slider_range, n_param):

    low, high = slider_range
    mask = (df_9['Station'] >= low) & (df_9['Station'] <= high)
    mask_22 = (df_22['Station'] >= low) & (df_22['Station'] <= high)
    mask_Amode22 = (df_Amode22['Station'] >= low) & (df_Amode22['Station'] <= high)

    if(n_param=='Nitrate[µmol/kg]'):  
        start_x_range= [0,35]
        color_str = 'NitrateQF'
    elif(n_param=='Oxygen[µmol/kg]'):
        start_x_range= [100,250]
        color_str = 'OxygenQF'
    elif(n_param=='CHLA[mg/m^3]'):
        start_x_range= [0, 0.25] 
        color_str = 'CHLAQF'
    elif(n_param=='BBP_700[1/m]'):
        start_x_range= [0, 0.004]
        color_str = 'BBPQF'

    fig = go.Figure()

    ##7901009##
    #inital fig made with plotly express, very important for html color toggles
    fig = px.scatter( df_9[mask], x=n_param, y='Depth[m]', color=color_str, 
                      color_discrete_sequence=line_colors_reds, hover_data=['Station'], range_x = start_x_range).update_traces(legendgroup= "7901009") 

    #looping profiles to add as traces
    trace_legend_bool = True
    for profile_num, df in df_9_grouped: 
        if(profile_num in range(low, high+1)): #+1 added to have range inclusive of 'high'
            trace_legend_bool = True if profile_num == low else False
            fig.add_trace(
                go.Scatter(
                    y= df['Depth[m]'], 
                    x= df[n_param],
                    name= "7901009 "+str(profile_num),
                    hoverinfo='skip',
                    legendgroup= "7901009",
                    line= dict(color= line_colors_reds[0], width= 1),
                    showlegend= trace_legend_bool) )
            
    ##4903622##
    #second fig made with plotly express, very important for html color toggles
    fig.add_trace( px.scatter(df_22[mask_22], x=n_param, y='Depth[m]', 
                   color_discrete_sequence=line_colors_blues, color=color_str, 
                   hover_data=['Station'], range_x = [0,35]).update_traces(legendgroup= "4903622", showlegend=False).data[0])

    #looping profiles to add traces
    trace_legend_bool = True
    for profile_num, df in df_22_grouped: 
        if(profile_num in range(low, high+1)): #+1 added to have range inclusive of 'high'
            trace_legend_bool = True if profile_num == low else False
            fig.add_trace(
                go.Scatter(
                    y= df['Depth[m]'], 
                    x= df[n_param],
                    name= "4903622 "+str(profile_num),
                    legendgroup= "4903622",
                    hoverinfo='skip',
                    line= dict(color= line_colors_blues[0], width= 1),
                    showlegend= trace_legend_bool) )

 ##4903622 A MODE##
#    #second fig made with plotly express, very important for html color toggles
    fig.add_trace( px.scatter(df_Amode22[mask_Amode22], x=n_param, y='Depth[m]', 
                   color_discrete_sequence=line_colors_greens, color=color_str, 
                   hover_data=['Station'], range_x = [0,35]).update_traces(legendgroup= "4903622_Amode", showlegend=True).data[0])

    #looping profiles to add traces
    trace_legend_bool = True
    for profile_num, df in df_Amode22_grouped: 
        if(profile_num in range(low, high+1)): #+1 added to have range inclusive of 'high'
            trace_legend_bool = True if profile_num == low else False
            fig.add_trace(
                go.Scatter(
                    y= df['Depth[m]'], 
                    x= df[n_param],
                    name= "4903622 "+str(profile_num),
                    legendgroup= "4903622_Amode",
                    hoverinfo='skip',
                    line= dict(color= line_colors_greens[0], width= 1),
                    showlegend= trace_legend_bool) )
    
    ##Formating##
    fig.update_layout(    
        yaxis= dict(
            domain= [0.05, 1],
            ticksuffix= " m",
            title=dict(text="Depth", font=dict(size=14,color ="#007FBF")),
            tickfont= dict(color= "#007FBF", size= 10),
            ticks="inside", 
            tickwidth=1,
            ticklen=10,
            tickcolor="#007FBF",
            autorange= "reversed",
            showgrid=True,
            fixedrange=False,
            zeroline= False
        ),
        xaxis= dict(                         
            title=dict( text=n_param, font=dict(size=14,color ="#007FBF")),
            tickfont= dict( color= "#007FBF", size= 10),
            showgrid=True,
            tickwidth=1,
            side= "top",
            ticks="inside", 
            tickcolor="#007FBF", 
            ticklen=10,
            fixedrange=False,
            zeroline= False 
        ),
        legend= dict( xanchor= "right", yanchor= "bottom", y= 0.1, x= .95,
                      font = dict( size = 8),
                      bgcolor="#F2F5F7",),
        #title_text= "<b>NIT_ADJUSTED</b>",
        plot_bgcolor = "#F2F5F7", #"#E6F7FF", # "#0085ca"
        #title_font_family= "Open Sans",
        title_font_size= 16,
        title_font_color= "#003087",
        title_x= 0.01,
        title_y= .99,
        width= 700,
        height= 700)
    
    #HTML export
    fig.write_html('testing1234.html')
    
    return fig

app.run_server(mode='external', debug=True)

In [ ]:
#fig.gettraces()

In [ ]:
def gen_Netcdf( floatNum, BRfileName, BDfileName):
    #Copying BRnetcdf data object to new working netcdf file
    os.system('cp ' + data_dir + floatNum+ '/' +  BRfileName + " " + data_dir + floatNum+ '/' + 'BD/'+BDfileName)
        ##Unix/Mac format os.system('cp file1.txt file7.txt')
        ##Windows format os.system('copy file1.txt file7.txt')
    
    #Opening working BDfile 
    ##format to open file in NETCDF3_CLASSIC
    ## 'a' (append) mode automatically reads previous netcdf format
    BDfile = netCDF4.Dataset(data_dir + floatNum+ '/' + 'BD/' + BDfileName, 'a') #opening BDfile
    return BDfile

In [ ]:
def set_DOXY_vars( BDfile, df_ODV, OxygenQC_index, sageGain):
    #setting DOXY_ADJUSTED, DOXY_ADJUSTED_QC, and DOXY_ADJUSTED_ERROR
    #initalizing empty arrays to write ODV data too
    
    DOXY_Adjusted_Array = np.ma.empty(shape = (BDfile.dimensions["N_LEVELS"].size), fill_value=99999.0, dtype='float32')
    DOXY_Adjusted_Array[:] = 99999.0
    DOXY_Adjusted_Array.mask = True
    
    DOXY_AdjustedQC_Array = np.ma.empty(shape = (BDfile.dimensions["N_LEVELS"].size), dtype='|S1')
    DOXY_AdjustedQC_Array[:] = ''
    DOXY_AdjustedQC_Array.mask = True

    DOXY_Adjusted_Error = np.ma.empty(shape = (BDfile.dimensions["N_LEVELS"].size), fill_value=99999.0, dtype='float32')
    DOXY_Adjusted_Error[:] = 99999.0
    DOXY_Adjusted_Error.mask = True
    
    
    for i in range(BDfile.dimensions["N_LEVELS"].size):
        #DOXY_Adjusted_Array
        x = np.float32( BDfile.variables["DOXY"][1, i] *np.float32(sageGain))
        if ~BDfile.variables["DOXY"][1,i].mask: 
            DOXY_Adjusted_Array[i] = x 
            #set DOXY_AdjustedQC_Array to 'Good'. or 1
            DOXY_AdjustedQC_Array[i] = 1

        #iterating through ODV file for selected cycle
        for row in df_ODV.index.values:
            if (np.float32(df_ODV.iloc[row]['Pressure[dbar]']) == np.round( BDfile.variables['PRES'][1,i], 2)):
                if ~(BDfile.variables["DOXY"][1,i].mask): 
                    DOXY_Adjusted_Error[i] = clim_adjusted_error_calc( pd.to_datetime(df_ODV.iloc[row]['mon/day/yr'], format= "%m/%d/%Y" ).to_julian_date(), 
                                                                          pd.to_datetime(df_ODV.iloc[row]['mon/day/yr'], format= "%m/%d/%Y" ).to_julian_date(),
                                                                          df_ODV.iloc[row]['Temperature[°C]'], 
                                                                          df_ODV.iloc[row]['Salinity[pss]'])
